<a href="https://colab.research.google.com/github/Merukkubus/BIS/blob/main/%D0%91%D0%98%D0%A1_%D0%9B%D0%914.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Класс TrithemusCipher (шифрование Тритемуса) (9)

In [ ]:
class TrithemusCipher:
    """Класс для работы с шифром Тритемуса."""
    ALPHABET = "_АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЫЬЭЮЯ"
    STEP = 8

    @staticmethod
    def thrithemus_table(key):
        """Создание таблицы Тритемуса на основе ключа."""
        out = ""
        for char in key:
            if char not in out:
                out += char
        for char in TrithemusCipher.ALPHABET:
            if char not in out:
                out += char
        return out

    @staticmethod
    def shift_trithemus(table, shift):
        """Сдвиг таблицы Тритемуса."""
        if shift > 0:
            s = table[31]
            head = table[:shift - 1]
            tail = table[shift - 1:31]
            table = head + s + tail
        return table

    @staticmethod
    def frw_S_Trithemus(block, key, j):
        """Прямое шифрование блока с использованием таблицы Тритbмуса."""
        out = "input error"
        if len(block) == 4:
            out = ""
            key_table = TrithemusCipher.thrithemus_table(key)
            jm = j % 32
            if jm > 0:
                for i in range(jm):
                    key_table = TrithemusCipher.shift_trithemus(key_table, i)

            for i in range(4):
                tmp = block[i]
                t = (j + i) % 32
                key_table = TrithemusCipher.shift_trithemus(key_table, t)
                pos = key_table.find(tmp)
                csym = key_table[(pos + TrithemusCipher.STEP) % 32]
                out += csym
        return out

    @staticmethod
    def inv_S_Trithemus(block, key, j):
        """Обратное шифрование блока с использованием таблицы Тритемуса."""
        out = "input error"
        if len(block) == 4:
            out = ""
            key_table = TrithemusCipher.thrithemus_table(key)
            jm = j % 32
            if jm > 0:
                for i in range(jm):
                    key_table = TrithemusCipher.shift_trithemus(key_table, i)

            for i in range(4):
                tmp = block[i]
                t = (j + i) % 32
                key_table = TrithemusCipher.shift_trithemus(key_table, t)
                pos = key_table.find(tmp)
                csym = key_table[(32 + pos - TrithemusCipher.STEP) % 32]
                out += csym
        return out

    @staticmethod
    def frw_S_TrithemusM(block, key, j):
        """Прямое шифрование блока с дополнительным улучшением."""
        tmp = TrithemusCipher.frw_S_Trithemus(block, key, j)
        return TrithemusCipher.frw_improve_block(tmp, key, j)

    @staticmethod
    def inv_S_TrithemusM(block, key, j):
        """Обратное шифрование блока с дополнительным улучшением."""
        tmp = TrithemusCipher.inv_improve_block(block, key, j)
        return TrithemusCipher.inv_S_Trithemus(tmp, key, j)

    @staticmethod
    def frw_improve_block(block, key, j):
        """Улучшение блока после прямого шифрования."""
        t = key
        while j > len(t) - 4:
            t += t

        key = t[j:j + 4]
        k = Utilities.text2array(key)
        b = Utilities.text2array(block)
        q = [(k[i] + k[(i + 1) % 4] + k[(i + 2) % 4] + k[(i + 3) % 4]) % 4 for i in range(4)]

        for i in range(3):
            j = (q[i] + i + 1) % 4
            l = (q[i] + i) % 4
            b[j] = (b[j] + b[l]) % 32

        return Utilities.array2text(b)

    @staticmethod
    def inv_improve_block(block, key, j):
        """Улучшение блока после обратного шифрования."""
        t = key
        while j > len(t) - 4:
            t += t

        key = t[j: j + 4]
        k = Utilities.text2array(key)
        b = Utilities.text2array(block)
        q = [(k[i] + k[(i + 1) % 4] + k[(i + 3) % 4]) % 32 for i in range(4)]

        for i in range(2, -1, -1):
            j = (q[i] + i + 1) % 4
            l = (q[i] + i) % 4
            b[j] = (b[j] - b[l]) % 32

        return Utilities.array2text(b)

    @staticmethod
    def oneside_trithemus(block_in, const_in, n_in):
        """Одностороннее шифрование с использованием шифра Тритемуса."""
        data = block_in
        key = const_in + data

        for i in range(n_in):
            data = TrithemusCipher.frw_S_TrithemusM(data, key, 0)
            key = data + TrithemusCipher.thrithemus_table(key)
        return data


# Класс Utilities (вспомогательные функции) (14)

In [ ]:
def trim_array(arr):
    for i, val in enumerate(arr):
        if val == "":
            return arr[:i]  # Срезаем массив до появления первой пустой строки
    return arr

def set_value_at_index(arr, index, value):
    if index >= len(arr):
        # Расширяем массив до нужного индекса
        arr.extend([0] * (index - len(arr) + 1))
    arr[index] = value

def submatrix(matrix, row_start, row_end, col_start, col_end):
    return [row[col_start:col_end + 1] for row in matrix[row_start:row_end + 1]]

In [ ]:
class Utilities:
    """Класс утилит для работы с символами и числами."""

    @staticmethod
    def sym2num(sym):
        """Преобразование символа в его числовое представление."""
        tmp = ord(sym)
        if tmp != 95:  # Если символ не является '_'
            out = tmp - 1039 - (1 if tmp > 1066 else 0)
        else:
            out = 0
        return out

    @staticmethod
    def num2sym(num):
        """Преобразование числа обратно в символ."""
        if num != 0:
            tmp = num + 1039 + (1 if num > 26 else 0)
            out = chr(tmp)
        else:
            out = "_"
        return out

    @staticmethod
    def text2array(text):
        """Преобразование текста в массив чисел."""
        return [Utilities.sym2num(char) for char in text]

    @staticmethod
    def array2text(array):
        """Преобразование массива чисел обратно в текст."""
        return "".join(Utilities.num2sym(num) for num in array)

    @staticmethod
    def block2num(block):
        """Преобразование блока текста в число."""
        # if len(block) != 4:
        #     return "input_error"
        out = 0
        for char in block:
            pos = TrithemusCipher.ALPHABET.index(char)
            out = out * 32 + pos
        return out

    @staticmethod
    def num2block(num):
        """Преобразование числа обратно в блок текста."""
        rem = num
        tmp = []
        for _ in range(4):
            tmp.append(TrithemusCipher.ALPHABET[rem % 32])
            rem = rem // 32
        return ''.join(tmp[::-1])

    @staticmethod
    def dec2bin(num_in):
        """Преобразование десятичного числа в двоичный массив длиной 20 бит."""
        rem = num_in
        out = [0] * 20
        for i in range(20):
            out[19 - i] = rem % 2
            rem = rem // 2
        return out

    @staticmethod
    def bin2dec(bin_in):
        """Преобразование двоичного массива обратно в десятичное число."""
        out = 0
        for bit in bin_in:
            out = 2 * out + bit
        return out

    @staticmethod
    def count_unity_bits(num_in):
        """Подсчёт количества единиц в двоичном представлении числа."""
        rem = num_in
        out = 0
        for _ in range(20):
            tmp = rem % 2
            rem = rem // 2
            out += tmp
        return out

    @staticmethod
    def compose_num(num1_in, num2_in, cont_in):
        """Склеивание двух чисел в зависимости от управляющего параметра."""
        if 0 < cont_in < 20:
            arr1 = Utilities.dec2bin(num1_in)
            arr2 = Utilities.dec2bin(num2_in)
            tmp = arr1[:cont_in] + arr2[cont_in:]
            out = int(''.join(map(str, tmp)), 2)
        elif cont_in == 0:
            out = num1_in
        else:
            out = num2_in
        return out

    @staticmethod
    def block2bin(block):
        """Преобразование блока текста в двоичный массив."""
        return Utilities.dec2bin(Utilities.block2num(block))

    @staticmethod
    def bin2block(bin_arr):
        """Преобразование двоичного массива обратно в блок текста."""
        return Utilities.num2block(Utilities.bin2dec(bin_arr))

    @staticmethod
    def push_reg(bin_in, bool_in):
        """Сдвиг регистра с добавлением нового значения."""
        n = len(bin_in) - 2
        out = [0] * len(bin_in)
        for i in range(n, 0, -1):
            out[i] = bin_in[i + 1]
        out[len(bin_in) - 1] = bool_in
        return out

    @staticmethod
    def taps2bin(taps_in):
        """Создание двоичного массива на основе входных касаний."""
        taps = sorted(taps_in, reverse=True)
        last = taps[0]
        y = 20 - last

        out = [0] * 20
        if y > 0:
            for i in range(y):
                out[i] = 0

        j = 0
        for i in range(last - 1):
            if last - i == taps[j]:
                out[y + i] = 1
                j += 1
            else:
                out[y + i] = 0
            if j > len(taps) - 1:
                break

        q = len(out)
        if q < last:
            for i in range(q, 20):
                out[i] = 0

        return out

## ПОШЛА ЛБ4

    @staticmethod
    def sym2bin(s_in):
        vec = [ord(char) for char in s_in]
        return vec[0] - 48

    @staticmethod
    def isSym(s_in):
        C = "АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЫЬЭЮЯ_"
        return 1 if s_in in C else -1

    @staticmethod # СЛАЙД 26 СХОД? (чуть чуть колхоза накинули)
    def msg2bin(MSG_IN):
        M = len(MSG_IN)
        i = 0
        f = 0
        tmp = [""] * (M * 5)
        while Utilities.isSym(MSG_IN[i]) == 1:
            p = MSG_IN[i]
            c = Utilities.sym2num(p)
            for j in range(5):
              index = i * 5 + 4 - j
              if index >= len(tmp):
                tmp.extend([0] * (index - len(tmp) + 1))
              if (c % 2) == 0:
                tmp[index] = 0
              else:
                tmp[index] = 1
              c //= 2

            if i == M - 1:
                f = 1
                break
            else:
              i += 1
        if f == 0:
          for k in range(i, M):
              p = MSG_IN[k]
              index = 4 * i + k
              if index >= len(tmp):
                tmp.extend([0] * (index - len(tmp) + 1))
              tmp[index] = Utilities.sym2bin(p)
        tmp = trim_array(tmp)

        return tmp

    @staticmethod # СЛАЙД 27 СХОД
    def bin2msg(BIN_IN):
      B = len(BIN_IN)
      b = B // 5
      q = B % 5
      out = ""

      for i in range(b):
          t = 0
          for j in range(5):
              t = 2 * t + BIN_IN[i * 5 + j]
          out += Utilities.num2sym(t)

      if q > 0:
          t = 0
          for k in range(1, q + 1):
              out += str(BIN_IN[b * 5 + k - 1])
      return out

    @staticmethod ## Есть в PacketHandler.prepare_packet, как реализовать хз
    def add_txt(template, data):
        """
        Добавляет текст к шаблону, заполняя его до нужной длины.
        :param template: Строка-шаблон фиксированной длины.
        :param data: Входная строка для вставки.
        :return: Заполненная строка.
        """
        # Вставляем данные в начало, заполняем оставшиеся символы '_'
        return data + template[len(data):]

    @staticmethod # Слайд 34
    def textxor(A_IN, B_IN):
        """
        Выполняет операцию XOR для двух входных текстов длиной 80 бит (по 4 символа в блоке).
        :param A_IN: Входной текст A.
        :param B_IN: Входной текст B.
        :return: Результирующий текст после применения XOR.
        """
        out = ""  # Инициализация выходной строки

        for i in range(4):
            # Извлекаем подстроки длиной 4 символа
            a = A_IN[i * 4: (i + 1) * 4]
            b = B_IN[i * 4: (i + 1) * 4]

            # Преобразуем блоки в числа
            A = Utilities.block2num(a)
            B = Utilities.block2num(b)

            # Преобразуем числа в двоичные массивы
            A_bin = Utilities.dec2bin(A)
            B_bin = Utilities.dec2bin(B)

            # XOR каждого бита блоков
            C_bin = [((A_bin[j] + B_bin[j]) % 2) for j in range(20)]

            # Преобразуем результат обратно в блок текста
            C = Utilities.bin2dec(C_bin)
            out += Utilities.num2block(C)

        return out

    @staticmethod # СЛАЙД 63
    def combine(STRSET_IN):
        out = ""
        for i in range(len(STRSET_IN)):
            out += STRSET_IN[i]
        return out

    @staticmethod
    def blockxor (A_IN, B_IN):
      a = Utilities.dec2bin(Utilities.block2num(A_IN))
      b = Utilities.dec2bin(Utilities.block2num(B_IN))
      c = [""] * 20
      for i in range(20):
        c[i] = (a[i] + b[i]) % 2
      c_bin = Utilities.bin2dec(c)
      return Utilities.num2block(c_bin)

# Класс BlockTransform (операции с блоками) (5)

In [ ]:
class BlockTransform:
    """Класс для преобразований блоков текста."""

    @staticmethod
    def subblocks_xor(blockA, blockB):
        """Побитовый XOR для подблоков."""
        binA = Utilities.block2bin(blockA)
        binB = Utilities.block2bin(blockB)
        binO = [(binA[j] + binB[j]) % 2 for j in range(len(binA))]
        return Utilities.bin2block(binO)

    @staticmethod
    def block_xor(blockA, blockB):
        """Побитовый XOR для всех подблоков."""
        nb = len(blockA) // 4
        out = ''
        for i in range(nb):
            subA = blockA[i * 4:(i + 1) * 4]
            subB = blockB[i * 4:(i + 1) * 4]
            out += BlockTransform.subblocks_xor(subA, subB)
        return out

    @staticmethod
    def binary_shift(array_in, shift_in):
        """Циклический сдвиг массива."""
        s = len(array_in)
        b = shift_in % s
        out = [0] * s
        if b > 0:
            for i in range(b, s):
                out[i] = array_in[i - b]
            for i in range(b):
                out[i] = array_in[s - b + i]
        else:
            for i in range(s + b):
                out[i] = array_in[i - b]
            for i in range(s + b, s):
                out[i] = array_in[i - s - b]
        return out

    @staticmethod
    def LB2B(block_in):
        """Преобразование длинного блока в двоичный массив."""
        out = [0] * 80
        for i in range(4):
            t = block_in[i * 4: i * 4 + 4]
            tmp = Utilities.dec2bin(Utilities.block2num(t))
            for j in range(20):
                out[j + i * 20] = tmp[j]
        return out

    @staticmethod
    def B2LB(block_in):
        """Преобразование двоичного массива обратно в длинный блок."""
        out = ''
        tmp = [0] * 20
        for i in range(4):
            for j in range(20):
                tmp[j] = block_in[j + i * 20]
            t = Utilities.num2block(Utilities.bin2dec(tmp))
            out += t
        return out


# Класс RoundKeys (генерация ключей и управление состоянием) (8)

In [ ]:
class RoundKeys:
    """Класс для генерации ключей и управления состоянием."""

    @staticmethod
    def lcg_next(state_in, coefs_in):
        """Генерация следующего числа с использованием линейного конгруэнтного генератора (LCG)."""
        a, c, m = coefs_in[0], coefs_in[1], coefs_in[2]
        return (a * state_in + c) % m

    @staticmethod
    def make_seed(block_in, os_fun):
        """Создание начального состояния (seed) на основе блока и функции OS."""
        str1 = "ПЕРВЫЙ_ГЕНЕРАТОР"
        str2 = "ВТОРОЙ_ГЕНЕРАТОР"
        str3 = "ТРЕТИЙ_ГЕНЕРАТОР"

        out_0 = os_fun(block_in, str1, 10)
        out_1 = os_fun(block_in, str2, 10)
        out_2 = os_fun(block_in, str3, 10)
        return [out_0, out_1, out_2]

    @staticmethod
    def seed2nums(array_in):
        """Преобразование массива блоков в числа."""
        return [Utilities.block2num(block) for block in array_in]

    @staticmethod
    def HCLCG_next(state_in, set_in):
        """Генерация следующего состояния и числа с использованием гибридного линейного конгруэнтного генератора."""
        first = RoundKeys.lcg_next(state_in[0], set_in[0])
        second = RoundKeys.lcg_next(state_in[1], set_in[1])
        control = RoundKeys.lcg_next(state_in[2], set_in[2])

        n = Utilities.count_unity_bits(control)

        if control % 2 == 0:
            out = Utilities.compose_num(first, second, n)
        else:
            out = Utilities.compose_num(second, first, n)

        return out, [first, second, control]

    @staticmethod
    def wrap_CHCLCG_next(init_flag, state_in, seed_in, os_fun, set_in):
        """Обёртка для CHCLCG генерации с инициализацией или обновлением состояния."""
        out = "something_wrong"
        stream = ""
        check = 0
        state = [0] * 4

        if init_flag == "up":
            for i in range(4):
                state[i] = RoundKeys.seed2nums(RoundKeys.make_seed(seed_in[i * 4: (i + 1) * 4], os_fun))
            check = 1
        elif init_flag == "down":
            state = state_in
            check = 1

        if check:
            for j in range(4):
                tmp = 0
                sign = 1
                for i in range(4):
                    T = RoundKeys.HCLCG_next(state[i], set_in)
                    state[i] = T[1]
                    tmp = (1048576 + sign * T[0] + tmp) % 1048576
                    sign = -sign
                stream += Utilities.num2block(tmp)
            out = (stream, state)

        return out

    @staticmethod
    def wrap_CHCLCGM_next(init_flag, state_in, seed_in, os_fun, set_in):
        """Обёртка для CHCLCG генерации с модифицированным состоянием."""
        out = "something_wrong"
        stream = ""
        check = 0
        state = []

        if init_flag == "up":
            seed = RoundKeys.check_seed(seed_in, os_fun)
            for i in range(4):
                state.append(RoundKeys.seed2nums(RoundKeys.make_seed(seed[i * 4:(i + 1) * 4], os_fun)))
                if i > 0:
                    for _ in range(i + 1):
                        T = RoundKeys.HCLCG_next(state[i], set_in)
                        state[i] = T[1]
            check = 1
        elif init_flag == "down":
            state = state_in
            check = 1

        if check:
            for j in range(4):
                tmp = 0
                sign = 1
                for i in range(4):
                    T = RoundKeys.HCLCG_next(state[i], set_in)
                    state[i] = T[1]
                    tmp = (1048576 + sign * T[0] + tmp) % 1048576
                    sign = -sign
                stream += Utilities.num2block(tmp)
            out = (stream, state)

        return out

    @staticmethod
    def check_seed(seed_in, os_fun):
        """Проверка и корректировка начального состояния."""
        c = "ОТВЕТСТВЕННЫЙ_ПОДХОД"
        t = []

        for i in range(4):
            t.append(seed_in[i * 4:(i + 1) * 4])

        for j in range(3):
            for i in range(j + 1, 4):
                if t[i] == t[j]:
                    t[i] = os_fun(t[j], c, j + 2 * i)

        return ''.join(t)

    @staticmethod
    def produce_round_keys(key, num_rounds, seed_function, rcg_function):
        """Генерация раундовых ключей."""
        out = [''] * num_rounds
        (out[0], state) = rcg_function("up", -1, key, seed_function, [[723482, 8677, 983609],
                                                                      [252564, 9109, 961193],
                                                                      [357630, 8971, 948209]])
        for i in range(1, num_rounds):
            (out[i], state) = rcg_function("down", state, -1, -1, [[723482, 8677, 983609],
                                                                   [252564, 9109, 961193],
                                                                   [357630, 8971, 948209]])
        return out

# Класс SPNetwork (SP-сеть) (8)

In [ ]:
class SPNetwork:
    """Класс для работы с SP-сетью."""

    MS1 = [[16, 3, 2, 13], [5, 10, 11, 8], [9, 6, 7, 12], [4, 15, 14, 1]]
    MS2 = [[7, 14, 4, 9], [12, 1, 15, 6], [13, 8, 10, 3], [2, 11, 5, 16]]
    MS3 = [[4, 14, 15, 1], [9, 7, 6, 12], [5, 11, 10, 8], [16, 2, 3, 13]]

    @staticmethod
    def frw_magic_square(block, matrix_index):
        """Прямое преобразование с использованием магического квадрата."""
        matrices = [SPNetwork.MS1, SPNetwork.MS2, SPNetwork.MS3]
        matrix = matrices[matrix_index]
        out = ""
        for row in matrix:
            for index in row:
                out += block[index - 1]
        return out

    @staticmethod
    def inv_magic_square(block, matrix_index):
        """Обратное преобразование с использованием магического квадрата."""
        matrices = [SPNetwork.MS1, SPNetwork.MS2, SPNetwork.MS3]
        matrix = matrices[matrix_index]
        tmp = [0] * 16
        for i in range(4):
            for j in range(4):
                tmp[matrix[i][j] - 1] = Utilities.sym2num(block[4 * i + j])
        return Utilities.array2text(tmp)

    @staticmethod
    def frw_round_SP(block, key, s_function, round_number):
        """Прямой SP-раунд."""
        inter = ""
        for i in range(4):
            sub_block = block[i * 4: i * 4 + 4]
            inter += s_function(sub_block, key, i * 4)
        tmp = SPNetwork.frw_magic_square(inter, round_number % 3)
        return BlockTransform.block_xor(tmp, key)

    @staticmethod
    def inv_round_SP(block, key, s_function, round_number):
        """Обратный SP-раунд."""
        tmp = BlockTransform.block_xor(block, key)
        inter = SPNetwork.inv_magic_square(tmp, round_number % 3)
        out = ""
        for i in range(4):
            sub_block = inter[i * 4: i * 4 + 4]
            out += s_function(sub_block, key, i * 4)
        return out

    @staticmethod
    def frw_P_round(block, r_in):
        """Прямое перестановочное преобразование."""
        r = r_in % 3
        j = 4 * (r_in % 4) + 2
        tmp = SPNetwork.frw_magic_square(block, r)
        shifted = BlockTransform.binary_shift(Utilities.text2array(tmp), j)
        return Utilities.array2text(shifted)

    @staticmethod
    def inv_P_round(block, r_in):
        """Обратное перестановочное преобразование."""
        r = r_in % 3
        j = -(4 * (r_in % 4) + 2)
        unshifted = BlockTransform.binary_shift(Utilities.text2array(block), j)
        return SPNetwork.inv_magic_square(Utilities.array2text(unshifted), r)

    @staticmethod
    def frw_SPNet(block_in, key_in, s_function, rcg_function, num_rounds):
        """Прямое шифрование с использованием SP-сети."""
        key_set = RoundKeys.produce_round_keys(key_in, num_rounds, s_function, rcg_function)
        block = block_in
        for i in range(num_rounds):
            block = SPNetwork.frw_round_SP(block, key_set[i], s_function, i)
        return block

    @staticmethod
    def inv_SPNet(block_in, key_in, s_function, rcg_function, num_rounds):
        """Обратное шифрование с использованием SP-сети."""
        key_set = RoundKeys.produce_round_keys(key_in, num_rounds, s_function, rcg_function)
        block = block_in
        is_function = TrithemusCipher.inv_S_TrithemusM
        for i in range(num_rounds - 1, -1, -1):
            block = SPNetwork.inv_round_SP(block, key_set[i], is_function, i)
        return block

    @staticmethod # СЛАЙД 61
    def enc_CTR(MSG_IN, IV_IN, KEY_IN, CIPHERFUN_SUBSET):
        """
        Реализация режима шифрования CTR (Counter Mode).
        :param MSG_IN: Входное сообщение для шифрования.
        :param IV_IN: Вектор инициализации (IV).
        :param KEY_IN: Ключ для шифрования.
        :param CIPHERFUN_SUBSET: Набор функций шифрования (CIPHER_FUN, SBLOCK_FUN).
        :return: Зашифрованное сообщение.
        """
        CIPHER_FUN = CIPHERFUN_SUBSET[0]
        SBLOCK_FUN = CIPHERFUN_SUBSET[1]
        r_in = CIPHERFUN_SUBSET[2]

        # Определение количества блоков
        m = len(MSG_IN) // 16
        IV_starter = IV_IN[:12]  # Первые 12 символов IV
        IV_ender = "____"  # Хвостовая часть IV
        ctr = 0  # Счётчик
        out = ""  # Итоговый результат

        for i in range(m):
            # Преобразуем счётчик в блок
            IV_ender = Utilities.num2block(ctr)

            # Конкатенируем IV
            IV = IV_starter + IV_ender

            # Генерация keystream с помощью CIPHER_FUN
            keystream = CIPHER_FUN(IV, KEY_IN, SBLOCK_FUN, r_in)

            # Извлечение текущего блока сообщения
            inp = MSG_IN[i * 16:(i + 1) * 16]

            # XOR сообщения с keystream и добавление к результату
            out += Utilities.textxor(inp, keystream)

            # Увеличение счётчика
            ctr += 1

        return out

    @staticmethod # СЛАЙД 62
    def mac_CBC(MSG_IN, IV_IN, KEY_IN, CIPHERFUN_SUBSET):
        """
        Реализация генерации MAC (Message Authentication Code) в режиме CBC (Cipher Block Chaining).
        :param MSG_IN: Входное сообщение для вычисления MAC.
        :param IV_IN: Вектор инициализации (IV).
        :param KEY_IN: Ключ для шифрования.
        :param CIPHERFUN_SUBSET: Набор функций шифрования (CIPHER_FUN, SBLOCK_FUN).
        :return: MAC (контрольная сумма).
        """
        CIPHER_FUN = CIPHERFUN_SUBSET[0]
        SBLOCK_FUN = CIPHERFUN_SUBSET[1]
        r_in = CIPHERFUN_SUBSET[2] # Распаковка функций

        # Определение количества блоков
        m = len(MSG_IN) // 16
        ctr = 0  # Счётчик блоков (не используется, но оставлен для расширяемости)
        out = ""  # Итоговый результат
        feedback = IV_IN  # Инициализация обратной связи

        for i in range(m):
            # Извлечение текущего блока сообщения
            inp = MSG_IN[i * 16: 16]

            # Применение XOR между обратной связью и текущим блоком
            temp = Utilities.textxor(feedback, inp)

            # Шифрование результата XOR
            feedback = CIPHER_FUN(temp, KEY_IN, SBLOCK_FUN, r_in)

            # Добавление шифрованного блока в результат
            out += feedback

        return feedback  # Возвращаем последний блок как MAC

    @staticmethod #СЛАЙД 64
    def CCM_frw(PACKET_IN, KEY_IN, onlymac, CIPHERFUN_SUBSET):
        """
        Реализация прямой CCM (Cipher Counter Mode) с флагом onlymac.
        :param PACKET_IN: Входной пакет (словарь с ASSDATA_IN, IV_IN, MSG_IN).
        :param KEY_IN: Ключ для шифрования.
        :param onlymac: Флаг (0 – шифрование и имитовставка, 1 – только имитовставка).
        :param CIPHERFUN_SUBSET: Набор функций шифрования (CIPHER_FUN, SBLOCK_FUN).
        :return: Пакет с полями ASSDATA_IN, IV_IN, MSG, MAC.
        """
        ASSDATA_IN = PACKET_IN[0]
        IV_IN = PACKET_IN[1]
        MSG_IN = PACKET_IN[2]
        tmp = PACKET_IN[3]

        # Объединяем связанные данные
        data = Utilities.combine(ASSDATA_IN)
        M = len(MSG_IN)

        # Генерация MAC
        mac = SPNetwork.mac_CBC(data + MSG_IN, IV_IN, KEY_IN, CIPHERFUN_SUBSET)

        if onlymac == 0:
            # Генерация зашифрованного сообщения с использованием MAC
            msg = SPNetwork.enc_CTR(MSG_IN + mac, IV_IN, KEY_IN, CIPHERFUN_SUBSET)
            MSG = msg[:M]  # Сообщение без MAC
            MAC = msg[M:]  # MAC
        else:
            MSG = MSG_IN
            MAC = mac

        # Формирование выходного пакета
        return [ASSDATA_IN, IV_IN, MSG, MAC]

    @staticmethod #65
    def CCM_inv(PACKET_IN, KEY_IN, onlymac, CIPHERFUN_SUBSET):
        """
        Реализация обратной CCM (Cipher Counter Mode) с флагом onlymac.
        :param PACKET_IN: Входной пакет (словарь с ASSDATA_IN, IV_IN, MSG_IN, MAC_IN).
        :param KEY_IN: Ключ для шифрования.
        :param onlymac: Флаг (0 – расшифрование и проверка, 1 – только проверка).
        :param CIPHERFUN_SUBSET: Набор функций шифрования (CIPHER_FUN, SBLOCK_FUN).
        :return: Пакет с полями ASSDATA_IN, IV_IN, MSG, MAC.
        """
        ASSDATA_IN = PACKET_IN[0]
        IV_IN = PACKET_IN[1]
        MSG_IN = PACKET_IN[2]
        MAC_IN = PACKET_IN[3]

        # Объединяем связанные данные
        data = Utilities.combine(ASSDATA_IN)
        M = len(MSG_IN)

        if onlymac == 0:
            # Расшифрование сообщения с использованием MAC
            msg = SPNetwork.enc_CTR(MSG_IN + MAC_IN, IV_IN, KEY_IN, CIPHERFUN_SUBSET)
            MSG = msg[:M]  # Сообщение без MAC
            MAC = msg[M:]  # MAC
        else:
            MSG = MSG_IN
            MAC = MAC_IN

        # Генерация и проверка MAC
        mac = SPNetwork.mac_CBC(data + MSG, IV_IN, KEY_IN, CIPHERFUN_SUBSET)
        MAC = Utilities.textxor(MAC, mac)

        # Формирование выходного пакета
        return [ASSDATA_IN, IV_IN, MSG, MAC]

    @staticmethod
    def block_SP_frw(block_in, key_in, s_fun, r_in):
      block = block_in
      for i in range(r_in):
        block = SPNetwork.frw_round_SP(block, key_in[i], s_fun, i)
      return block

    @staticmethod
    def block_SP_inv(block_in, key_in, s_fun, r_in):
      block = block_in
      is_fun = TrithemusCipher.inv_S_TrithemusM
      for i in range(r_in - 1, -1, -1):
        block = SPNetwork.inv_round_SP(block, key_in[i], is_fun, i)
      return block


In [ ]:
out = SPNetwork.frw_SPNet('КОРЫСТЬ_СЛОНА_ЭХ', 'МТВ_ВСЕ_ЕЩЕ_ТЛЕН', TrithemusCipher.frw_S_TrithemusM, RoundKeys.wrap_CHCLCGM_next, 8)
print(out)


НЩЛДХМЕВАЭЮТЙЖУЗ


# Класс PaddingHandler

In [ ]:
class PaddingHandler:
    @staticmethod # СЛАЙД 29
    def check_padding(BINMSG_IN):
        BINS = BINMSG_IN
        M = len(BINS)
        blocks = M // 80
        remainder = M % 80

        numblocks = 0
        padlength = 0

        if remainder == 0:
            tb = BINS[M - 20 : M]
            ender = tb[17:20]
            #print("check_padding: последние 20 бит (tb) =", tb)
            #print("check_padding: последние 3 бита (ender) =", ender)

            if all(ender[i] == [0, 0, 1][i] for i in range(3)):
                NB = tb[7:17]
                PL = tb[0:7]
                padlength = 0

                for i in range(7):
                    padlength = 2 * padlength + PL[i]
                    #print(f"padlength бит {i}: {PL[i]}, текущее значение: {padlength}")

                numblocks = 0
                for i in range(10):
                    numblocks = 2 * numblocks + NB[i]
                    #print(f"numblocks бит {i}: {NB[i]}, текущее значение: {numblocks}")

                if (numblocks == blocks) and (23 <= padlength < 103):
                    tb = BINS[M - padlength: M - 20]
                    starter = tb[0]
                    f = 0

                    if starter == 1:
                        f = 1
                        for j in range(1, padlength):
                            tmp = tb[j * 80:(j + 1) * 80]
                            if any(tmp):
                                f = 0
                                break
                    else:
                        f = 0
                else:
                    f = 0
            else:
                f = 0
        else:
            f = 0

        tb = BINS[-20:]
        #print(f"check_padding: извлеченные последние 20 бит (tb) = {tb}")
        ender = tb[-3:]
        #print(f"check_padding: последние 3 бита из tb (ender) = {ender}")
        #print("check_padding: f =", f, "numblocks =", numblocks, "padlength =", padlength)
        return f, [numblocks, padlength]

    @staticmethod # СЛАЙД 29
    def produce_padding(rem_in, blocks_in):

        if rem_in == 0:
            b = blocks_in + 1
            r = 80
        elif rem_in <= 57:
            r = 80 - rem_in
            b = blocks_in + 1
        else:
            r = 160 - rem_in
            b = blocks_in + 2

        pad = [0] * r
        pad[0] = 1
        for i in range(1, r - 21):
            pad[i] = 0

        rt = r
        for i in range(6, -1, -1):
            pad[r - 20 + i] = rt % 2
            rt //= 2

        for i in range(9, -1, -1):
            pad[r - 13 + i]= b % 2
            b //= 2

        pad[-3:] = [0, 0, 1]
        #pad = trim_array(pad)
        #print("produce_padding: pad =", pad)
        #print(f"produce_padding: закодированное количество блоков (b) = {b}")
        #print(f"produce_padding: закодированная длина паддинга (r) = {r}")
        return pad

    @staticmethod # СЛАЙД 30 (в пунктах описания 2 и 3 неправильно указано количество битов, должно быть 7 и 10) где???
    def pad_message(MSG_IN):
      pad = []
      BINS = Utilities.msg2bin(MSG_IN)
      M = len(BINS)
      blocks = M // 80
      remainder = M % 80

      #print("pad_message: M =", M, "blocks =", blocks, "remainder =", remainder)

      if remainder == 0:
          f = PaddingHandler.check_padding(BINS)[0]
      else:
          f = 1

      if f == 1:
          pad = PaddingHandler.produce_padding(remainder, blocks)
          BINS.extend(pad)
      #     for j in range(len(pad)):
      #         if (M + j) < len(BINS):
      #           BINS[M + j] = pad[j]
      #         else:
      #           BINS.extend([""] * (M + j - len(BINS) + 1))
      #           BINS[M + j] = pad[j]
      # BINS = trim_array(BINS)
      #print("pad_message: final BINS length =", len(BINS))
      return Utilities.bin2msg(BINS)

    @staticmethod # СЛАЙД 30
    def unpad_message(MSG_IN):
        BINS = Utilities.msg2bin(MSG_IN)
        M = len(BINS)
        T = PaddingHandler.check_padding(BINS)

        if T[0] == 1:
            pl = T[1][1]
            # tmp = submatrix(BINS, 0, M - pl - 1, 0, 0)
            tmp = BINS[:M - pl]
            out = Utilities.bin2msg(tmp)
        else:
            out = MSG_IN
        return out

# Класс PacketHandler

In [ ]:
class PacketHandler:
    @staticmethod
    def prepare_packet(DATA_IN, IV_in, MSG_IN):
        """
        Подготовка пакета данных.
        :param DATA_IN: Входные данные.
        :param IV_in: Входной вектор инициализации.
        :param MSG_IN: Сообщение.
        :return: Сформированный пакет.
        """
        data = DATA_IN
        iv_template = "________"  # Убедитесь, что шаблон совпадает по длине с IV_in

        iv = Utilities.add_txt(iv_template, IV_in)  # Добавляем текстовый IV
        msg = PaddingHandler.pad_message(MSG_IN)  # Пакуем сообщение
        L = len(Utilities.msg2bin(msg))  # Длина сообщения в битах
        a = "________"  # Инициализация 8-символьного идентификатора

        # Генерация идентификатора на основе длины
        for i in range(5):
            a = Utilities.num2sym(L % 32) + a  # Конкатенация символов
            L //= 32

        data = a  # Перезапись данных идентификатором
        mac = "________"  # Заглушка для MAC

        out = {
            "data": data,
            "iv": iv,
            "msg": msg,
            "mac": mac,
        }
        return out

    @staticmethod
    def validate_packet(PACKET_IN):
        """
        Проверка пакета данных.
        :param PACKET_IN: Входной пакет.
        :return: Корректность пакета (1 – корректно, 0 – некорректно).
        """
        data = PACKET_IN["data"]
        iv = PACKET_IN["iv"]
        msg = PACKET_IN["msg"]
        mac = PACKET_IN["mac"]

        f = 1  # Флаг корректности
        t = data[0]  # Первый символ
        s = data[1]  # Второй символ
        ml = len(mac)  # Длина MAC

        if t != "Б":  # Проверка заголовка
            f = 0
        elif (s == "A" or s == "Б") and (ml != 16):  # Проверка MAC длины
            f = 0
        elif s == "_" and ml != 0:  # Проверка пустого MAC
            f = 0

        return f

    @staticmethod
    def transmit(PACKET_IN):
        """
        Имитирует передачу пакета через линию связи, объединяя данные в битовый поток.
        :param PACKET_IN: Входной пакет (словарь с полями: data, iv, msg, mac).
        :return: Битовый поток.
        """
        # Конкатенация полей пакета
        out = (
            PACKET_IN["data"]
            + PACKET_IN["iv"]
            + PACKET_IN["msg"]
            + PACKET_IN["mac"]
        )
        # Преобразование в бинарный поток
        return Utilities.msg2bin(out)

    @staticmethod
    def receive(STREAM_IN):
        """
        Имитирует приём пакета, декодируя битовый поток обратно в поля пакета.
        :param STREAM_IN: Входной поток битов.
        :return: Словарь пакета с полями: type, sender, receiver, session, length, iv, message, mac.
        """
        p = Utilities.bin2msg(STREAM_IN)  # Преобразуем поток в текст
        M = len(p)

        # Извлечение полей пакета
        packet_type = p[0:2]
        sender = p[2:10]
        receiver = p[10:18]
        session = p[18:27]
        length_field = p[27:32]
        iv = p[32:48]

        # Декодирование длины сообщения
        L = 0
        for i in range(5):
            t = Utilities.sym2num(length_field[i])
            L = 32 * L + t

        # Извлечение сообщения и MAC
        message = p[48:48 + L]
        mac = p[48 + L:M]

        # Формирование выходного пакета
        return {
            "type": packet_type,
            "sender": sender,
            "receiver": receiver,
            "session": session,
            "length": length_field,
            "iv": iv,
            "message": message,
            "mac": mac,
        }



# Класс CCM

In [ ]:
def CCM(ASS_DATA, MSG_ARRAY, KEY_IN, CIPHER_SUITE, nonce, typ):
    # Распаковка ASS_DATA
    mtype = ASS_DATA[0]
    sender = ASS_DATA[1]
    receiver = ASS_DATA[2]
    transmission = ASS_DATA[3]

    # Распаковка CIPHER_SUITE
    RCG = CIPHER_SUITE[0][0]
    rcg_rounds = CIPHER_SUITE[0][1]

    Ciph_frw = CIPHER_SUITE[1][0][0]
    Ciph_inv = CIPHER_SUITE[1][0][1]
    ciph_rounds = CIPHER_SUITE[1][1]

    SB_frw = CIPHER_SUITE[2][0]
    SB_inv = CIPHER_SUITE[2][1]

    notimportant = CIPHER_SUITE[3]

    # Формирование IV0
    t1 = receiver + sender
    t2 = mtype + transmission + "_____"
    t3 = Utilities.add_txt(Utilities.add_txt(t1, t2), nonce)
    IV0 = t3[0:8] + t3[12:16] + t3[12:16]

    # Счетчик сообщений
    msg_counter = -1
    keyset = RoundKeys.produce_round_keys(KEY_IN, rcg_rounds, SB_frw, RCG)

    # Если отправка
    if typ == "send":
        out = []
        for i in range(len(MSG_ARRAY)):
            msg_sec = mtype
            msg_counter += 1
            IV1 = "________" + Utilities.num2block(msg_counter) + "____"
            IV = Utilities.textxor(IV0, IV1)

            tmp_packet = PacketHandler.prepare_packet(
                [msg_sec, sender, receiver, transmission], IV, MSG_ARRAY[i]
            )

            if msg_sec == "В_":
                out.append(PacketHandler.transmit(tmp_packet)) # [i]?

            elif msg_sec == "ВА":
                sec_packet = SPNetwork.CCM_frw(tmp_packet, keyset, 1, [Ciph_frw, SB_frw, ciph_rounds])
                out.append(PacketHandler.transmit(sec_packet)) # [i]?

            elif msg_sec == "ВБ":
                sec_packet = SPNetwork.CCM_frw(tmp_packet, keyset, 0, [Ciph_frw, SB_frw, ciph_rounds])
                out.append(PacketHandler.transmit(sec_packet)) # [i]?

        return out

    # Если прием
    elif typ == "receive":
        last = -1
        out = []
        for i in range(len(MSG_ARRAY)):
            tmp_packet = PacketHandler.receive(MSG_ARRAY[i])
            rdata = tmp_packet[0]
            x1 = tmp_packet[1][12:16]
            x2 = tmp_packet[1][8:12]

            current = Utilities.block2num(Utilities.blockxor(x1, x2))
            if current > last:
                if rdata[0] == "ВБ":
                    rec_packet = SPNetwork.CCM_inv(tmp_packet, keyset, 0, [Ciph_frw, SB_frw, ciph_rounds])
                    rec_packet[2] = PaddingHandler.unpad_message(rec_packet[2])
                    if rec_packet[3] == "________________":
                      last = current
                      rec_packet[3] = "OK"

                elif rdata[0] == "ВА" & mtype != "ВБ":
                    rec_packet = SPNetwork.CCM_inv(tmp_packet, keyset, 1, [Ciph_frw, SB_frw, ciph_rounds])
                    rec_packet[2] = PaddingHandler.unpad_message(rec_packet[2])
                    if rec_packet[3] == "________________":
                      last = current
                      rec_packet[3] = "OK"

                elif rdata[0] == "В_" and mtype == "ВА":
                    rec_packet = tmp_packet
                    rec_packet[2] = PaddingHandler.unpad_message(rec_packet[2])
                    if rec_packet[3] == "":
                      last = current
                      rec_packet[3] = "N/A"

                else:
                    rec_packet = tmp_packet
                out.append(rec_packet)

        return out


In [ ]:
def CCMFASF(ASS_DATA, MSG_ARRAY, KEY_IN, CIPHER_SUITE, nonce, typ):
  #распакуем ass_data
  mtype = ASS_DATA[0]
  sender = ASS_DATA[1]
  reciever = ASS_DATA[2]
  transmission = ASS_DATA[3]

  #распакуем cipher_suite
  # suite: | [wrap, 8]; [ (sp_frw, sp_inv), 8 ]; [trit_frw, trit_inv]; [ccmp_frw, ccmp_inv] |
  RCG = CIPHER_SUITE[0][0]
  rcg_rounds = CIPHER_SUITE[0][1]

  Ciph_frw = CIPHER_SUITE[1][0][0]
  Ciph_inv = CIPHER_SUITE[1][0][1]
  ciph_rounds = CIPHER_SUITE[1][1]

  SB_frw = CIPHER_SUITE[2][0]
  SB_inv = CIPHER_SUITE[2][1]

  notimportant = CIPHER_SUITE[3]

  t1 = reciever + sender
  t2 = mtype + transmission + "_____"
  t3 = Utilities.add_txt(Utilities.add_txt(t1, t2), nonce)

  IV0 = t3[0:8] + t3[12:16] + t3[12:16] #?

  msg_counter = -1
  keyset = RoundKeys.produce_round_keys(KEY_IN, rcg_rounds, SB_frw, RCG)

  if(typ == "send"):
    for i in range(len(MSG_ARRAY)):
      msg_sec = mtype
      msg_counter += 1
      IV1 = "________" + Utilities.num2block(msg_counter) + "____"
      IV = Utilities.textxor(IV0, IV1)
      tmp_packet = PacketHandler.prepare_packet([msg_sec, sender, reciever, transmission], IV, MSG_ARRAY[i])

      if(msg_sec == "В_"):
        out[i] = PacketHandler.transmit(tmp_packet)

      if(msg_sec == "ВА"):
        sec_packet = SPNetwork.CCM_frw(tmp_packet, keyset, 1, [Ciph_frw, SB_frw, ciph_rounds])
        out[i] = PacketHandler.transmit(sec_packet)

      if(msg_sec == "ВБ"):
        sec_packet = SPNetwork.CCM_frw(tmp_packet, keyset, 0, [Ciph_frw, SB_frw, ciph_rounds])
        out[i] = PacketHandler.transmit(sec_packet)

  if(typ == "recieve"):
    last = -1
    for i in range(len(MSG_ARRAY)):
      tmp_packet = PacketHandler.receive(MSG_ARRAY[i])
      rdata = tmp_packet[0]
      x1 = tmp_packet[1][12:16]
      x2 = tmp_packet[1][8:12]

      current = Utilities.block2num(Utilities.blockxor(x1,x2))
      if(current > last):
        if(rdata[0] == "ВБ"):
          rec_packet = SPNetwork.CCM_inv(tmp_packet, keyset, 0, [Ciph_frw, SB_frw, ciph_rounds])
          rec_packet[2] = PaddingHandler.unpad_message(rec_packet[2])
          if(rec_packet == "________________"):
            last = current
            rec_packet[3] = "ОК"
          elif(rdata[0] == "ВА") and (mtype != "ВБ"):
            rec_packet = SPNetwork.CCM_inv(tmp_packet, keyset, 1, [Ciph_frw, SB_frw, ciph_rounds])
            rec_packet[2] = PaddingHandler.unpad_message(rec_packet[2])



In [ ]:
suite = make_suite("Trithemus", "LCG", "SP Net", "CCM")
suite[2][1]

<function __main__.TrithemusCipher.inv_S_TrithemusM(block, key, j)>

# ЛБ4 CCM (CTR+CBC-MAC)


In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_root = 'drive/MyDrive/LB4 CS'
print(os.listdir(data_root))

['inp.txt', 'ad.txt']


In [ ]:
inp_path = os.path.join(data_root, 'inp.txt')
ad_path = os.path.join(data_root, 'ad.txt')

# Чтение данных из файлов и создание массивов
with open(inp_path, 'r', encoding='utf-8') as inp_file:
    INPUTS_ARRAY = [line.strip() for line in inp_file.readlines()]

with open(ad_path, 'r', encoding='utf-8') as ad_file:
    ASSOCDATA_ARRAY = [line.strip() for line in ad_file.readlines()]

print(f'ASSCODATA: {ASSOCDATA_ARRAY}')
print(f'INPUTS: {INPUTS_ARRAY}')
print(len(INPUTS_ARRAY[1]))

ASSCODATA: ['"ВА" "АЛИСА__А" "БОБ____А" "КОТОПОЕЗД"', '"ВБ" "АЛИСА_АЖ" "БОБ___ОЧ" "ЕГИПТЯНИН"', '"В_" "АЛИСА_ЯЗ" "БОБ___ЬЬ" "ЩЕГОЛЯНИЕ"', '"ВБ" "БОБ___ЬЬ" "АЛИСА_ЯЗ" "ЭКЛАМПСИЯ"', '"ВБ" "БОБ___ЬЬ" "АЛИСА_ЯЗ" "ЕГИПТЯНИН"', '"ВБ" "АЛИСА_ЯЗ" "БОБ___ЬЬ" "ЕГИПТЯНИН"']
INPUTS: ['ГАРРИ_С_ОТКРЫТЫМ_РТОМ_СМОТРЕЛ_НА_СЕМЕЙНОЕ_ХРАНИЛИЩЕ_ТЧК_У_НЕГО_БЫЛО_ТАК_МНОГО_ВОПРОСОВ_ЗПТ_ЧТО_ОН_ДАЖЕ_НЕ_ЗНАЛ_ЗПТ_С_КАКОГО_ИМЕННО_НАЧАТЬ_ТЧК_МАКГОНАГАЛЛ_СТОЯЛА_У_ДВЕРИ_И_НАБЛЮДАЛА_ЗА_МАЛЬЧИКОМ_ТЧК_ОНА_НЕБРЕЖНО_ОПИРАЛАСЬ_О_СТЕНУ_ЗПТ_НО_ВЗГЛЯД_У_НЕЕ_БЫЛ_НАПРЯЖЕННЫЙ_ТЧК_И_НЕСПРОСТА_ТЧК_ОКАЗАТЬСЯ_ПЕРЕД_ОГРОМНОЙ_КУЧЕЙ_ЗОЛОТЫХ_МОНЕТ_ТИРЕ_ТА_ЕЩЕ_ПРОВЕРКА_НА_ПРОЧНОСТЬ_ТЧК_', 'ИСТОРИЯ_ГАНСА_КАСТОРПА_ЗПТ_КОТОРУЮ_МЫ_ХОТИМ_ЗДЕСЬ_РАССКАЗАТЬ_ЗПТ_ТИРЕ_ОТНЮДЬ_НЕ_РАДИ_НЕГО_ПОСКОЛЬКУ_ЧИТАТЕЛЬ_В_ЕГО_ЛИЦЕ_ПОЗНАКОМИТСЯ_ЛИШЬ_С_САМЫМ_ОБЫКНОВЕННЫМ_ЗПТ_ХОТЯ_И_ПРИЯТНЫМ_МОЛОДЫМ_ЧЕЛОВЕКОМ_ЗПТ_ТИРЕ_ИЗЛАГАЕТСЯ_РАДИ_САМОЙ_ЭТОЙ_ИСТОРИИ_ЗПТ_ИБО_ОНА_КАЖЕТСЯ_НАМ_В_ВЫСОКОЙ_СТЕПЕНИ_ДОСТОЙНОЙ_ОПИСАНИЯ_ПРИЧЕМ_ЗПТ_К_ЧЕСТИ_ГАНСА_КАСТОРПА_ЗПТ_СЛЕДУЕТ_ОТМЕТИ

In [ ]:
# Провекра sym2bin
print(Utilities.sym2bin("1"))
print(Utilities.sym2bin("0"))

1
0


In [ ]:
from itertools import count
TEST = "ГНОЛЛЫ_ПИЛИЛИ_ПЫЛЕСОС_ЛОСОСЕМ"
TEST_1 = "ГНОЛЛЫ_ПИЛИЛИ_ПЫЛЕСОС_ЛОСОСЕМ0011"
TEST_2 = "ГНОЛЛЫ_ПИЛИЛИ_ПЫЛЕСОС_ЛОСОСЕМ1110011011011"

print(f"len(TEST) = {len(TEST)}")
print(f"len(TEST_1) = {len(TEST_1)}")
print(f"len(TEST_2) = {len(TEST_2)}")

Q = Utilities.msg2bin(TEST)
Q1 = Utilities.msg2bin(TEST_1)
Q2 = Utilities.msg2bin(TEST_2)

print("Q:", Q)
print("Q1:", Q1)
print("Q2:", Q2)

print("cols(Q) =", len(Q))
print("cols(Q1) =", len(Q1))
print("cols(Q2) =", len(Q2))

len(TEST) = 29
len(TEST_1) = 33
len(TEST_2) = 42
Q: [0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1]
Q1: [0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1]
Q2: [0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0

In [ ]:
# Генерация бинарных представлений
# сход
TEST = "ГНОЛЛЫ_ПИЛИЛИ_ПЫЛЕСОС_ЛОСОСЕМ"
TEST_1 = "ГНОЛЛЫ_ПИЛИЛИ_ПЫЛЕСОС_ЛОСОСЕМ00111"
TEST_2 = "ГНОЛЛЫ_ПИЛИЛИ_ПЫЛЕСОС_ЛОСОСЕМ1110011011011"

X = Utilities.msg2bin(TEST)
Y = Utilities.bin2msg(X)

X1 = Utilities.msg2bin(TEST_1)
Y1 = Utilities.bin2msg(X1)

X2 = Utilities.msg2bin(TEST_2)
Y2 = Utilities.bin2msg(X2)

# Сравнение результатов
print("Y:", Y)
print("Y1:", Y1)
print("Y2:", Y2)

Y: ГНОЛЛЫ_ПИЛИЛИ_ПЫЛЕСОС_ЛОСОСЕМ
Y1: ГНОЛЛЫ_ПИЛИЛИ_ПЫЛЕСОС_ЛОСОСЕМЖ
Y2: ГНОЛЛЫ_ПИЛИЛИ_ПЫЛЕСОС_ЛОСОСЕМЬЫ011


In [ ]:
# сход
l = len(INPUTS_ARRAY)
tmp = []
for i in range(l):
  tmp.append(len(Utilities.msg2bin(INPUTS_ARRAY[i])))
print(tmp)

[1840, 15395, 2595, 138, 58, 1555]


In [ ]:
# СЛАЙД 31 ПРОВЕРКА ПАДИНГОВ
#СХОД
TST1 = PaddingHandler.pad_message(INPUTS_ARRAY[0])
print("msg2bin(TST1) length:", len(Utilities.msg2bin(TST1)))
print("msg2bin(INPUTS_ARRAY[0]) length:", len(Utilities.msg2bin(INPUTS_ARRAY[0])))
print("Check padding:", PaddingHandler.check_padding(Utilities.msg2bin(TST1)))

print(".....")

TST1_1 = PaddingHandler.unpad_message(TST1)
print("msg2bin(TST1_1) length:", len(Utilities.msg2bin(TST1_1)))
print("TST1_1 = INPUTS_ARRAY[0]:", TST1_1.__eq__(INPUTS_ARRAY[0]))
print("TST1_1 = TST1:", TST1_1.__eq__(TST1))

print("----------------------------------------------------------------------------------------")
#СХОД
TST2 = PaddingHandler.pad_message(INPUTS_ARRAY[1])
print("msg2bin(INPUTS_ARRAY[1]) length:", len(Utilities.msg2bin(INPUTS_ARRAY[1])))
print("msg2bin(TST2) length:", len(Utilities.msg2bin(TST2)))
print("Check padding TST2:", PaddingHandler.check_padding(Utilities.msg2bin(TST2)))

print(".....")

TST2_1 = PaddingHandler.pad_message(TST2)
print("msg2bin(TST2_1) length:", len(Utilities.msg2bin(TST2_1)))
print("Check padding TST2_1:", PaddingHandler.check_padding(Utilities.msg2bin(TST2_1)))

print(".....")

TST2_2 = PaddingHandler.unpad_message(TST2_1)
print("msg2bin(TST2_2) length:", len(Utilities.msg2bin(TST2_2)))
print("TST2_2 = TST2:", TST2_2.__eq__(TST2))

print(".....")

TST2_3 = PaddingHandler.unpad_message(TST2_2)
print("msg2bin(TST2_3) length:", len(Utilities.msg2bin(TST2_3)))
print("TST2_3 = INPUTS_ARRAY[1]:", TST2_3.__eq__(INPUTS_ARRAY[1]))

print("----------------------------------------------------------------------------------------")
#СХОД
TST3 = PaddingHandler.pad_message(INPUTS_ARRAY[2])
print("msg2bin(TST3) length:", len(Utilities.msg2bin(TST3)))
print("msg2bin(INPUTS_ARRAY[2]) length:", len(Utilities.msg2bin(INPUTS_ARRAY[2])))
print("Check padding TST3:", PaddingHandler.check_padding(Utilities.msg2bin(TST3)))

print(".....")

TST3_1 = PaddingHandler.pad_message(TST3)
print("msg2bin(TST3_1) length:", len(Utilities.msg2bin(TST3_1)))

print(".....")

TST3_2 = PaddingHandler.unpad_message(TST3_1)
print("msg2bin(TST3_2) length:", len(Utilities.msg2bin(TST3_2)))
print("TST3_2 = TST3:", TST3_2.__eq__(TST3))

print(".....")

TST3_3 = PaddingHandler.unpad_message(TST3_2)
print("msg2bin(TST3_3) length:", len(Utilities.msg2bin(TST3_3)))
print("TST3_3 = INPUTS_ARRAY[2]:", TST3_3.__eq__(INPUTS_ARRAY[2]))
print("msg2bin(TST3_3) = msg2bin(INPUTS_ARRAY[2]):", Utilities.msg2bin(TST3_3).__eq__(Utilities.msg2bin(INPUTS_ARRAY[2])))

print("----------------------------------------------------------------------------------------")
#СХОД
TST4 = PaddingHandler.pad_message(INPUTS_ARRAY[3])
print("msg2bin(TST4) length:", len(Utilities.msg2bin(TST4)))
print("msg2bin(INPUTS_ARRAY[3]) length:", len(Utilities.msg2bin(INPUTS_ARRAY[3])))

msg2bin(TST1) length: 1840
msg2bin(INPUTS_ARRAY[0]) length: 1840
Check padding: (0, [0, 0])
.....
msg2bin(TST1_1) length: 1840
TST1_1 = INPUTS_ARRAY[0]: True
TST1_1 = TST1: True
----------------------------------------------------------------------------------------
msg2bin(INPUTS_ARRAY[1]) length: 15395
msg2bin(TST2) length: 15440
Check padding TST2: (1, [193, 45])
.....
msg2bin(TST2_1) length: 15520
Check padding TST2_1: (1, [194, 80])
.....
msg2bin(TST2_2) length: 15440
TST2_2 = TST2: True
.....
msg2bin(TST2_3) length: 15395
TST2_3 = INPUTS_ARRAY[1]: True
----------------------------------------------------------------------------------------
msg2bin(TST3) length: 2640
msg2bin(INPUTS_ARRAY[2]) length: 2595
Check padding TST3: (1, [33, 45])
.....
msg2bin(TST3_1) length: 2720
.....
msg2bin(TST3_2) length: 2640
TST3_2 = TST3: True
.....
msg2bin(TST3_3) length: 2595
TST3_3 = INPUTS_ARRAY[2]: True
msg2bin(TST3_3) = msg2bin(INPUTS_ARRAY[2]): True
------------------------------------------

In [ ]:
A1 = "ГОЛОВКА_КРУЖИТСЯ" #Слайд 34 сход
A2 = "МЫШКА_БЫЛА_ЛИХОЙ"
B1 = "СИНЕВАТАЯ_БОРОДА"
B2 = "ЗЕЛЕНЫЙ_КОТОЗМИЙ"

# Применение textxor
C1 = Utilities.textxor(A1, A2)
C2 = Utilities.textxor(A1, B2)

# Проверка обратного преобразования
C11 = Utilities.textxor(C1, A2)
C12 = Utilities.textxor(C1, A1)

print("C1:", C1)
print("C2:", C2)
print("C11 (A1 восстановлен):", C11)
print("C12 (A2 восстановлен):", C12)


C1: ИУФГБКВЫЖПУК_ДЭФ
C2: ЛИ_ИМПК__ЮЖЗАЮЫФ
C11 (A1 восстановлен): ГОЛОВКА_КРУЖИТСЯ
C12 (A2 восстановлен): МЫШКА_БЫЛА_ЛИХОЙ


In [ ]:
## СЛАЙД 35
def CCMP_frw():
  ...

def CCMP_inv():
  ...

def make_suite(S, R, C, M):
    """
    Генерирует пакет функций на основе заданных параметров.
    :param S: Тип шифрования Trithemus.
    :param R: Генератор ключей LCG.
    :param C: Тип преобразования SP Net.
    :param M: Режим работы CCM.
    :return: Набор функций [a1, a2, a3, a4].
    """
    # Выбор шифрования
    if S == "Trithemus":
        a3 = [TrithemusCipher.frw_S_TrithemusM, TrithemusCipher.inv_S_TrithemusM]
    else:
        raise ValueError(f"Unknown S parameter: {S}")

    # Выбор преобразования
    if C == "SP Net":
        a2 = [[SPNetwork.block_SP_frw, SPNetwork.block_SP_inv], 8]
    else:
        raise ValueError(f"Unknown C parameter: {C}")

    # Выбор генератора ключей
    if R == "LCG":
        a1 = [RoundKeys.wrap_CHCLCGM_next , 8]
    else:
        raise ValueError(f"Unknown R parameter: {R}")

    # Выбор режима работы
    if M == "CCM":
        a4 = [CCMP_frw, CCMP_inv]
    else:
          raise ValueError(f"Unknown M parameter: {M}")

    # Формируем пакет функций
    return [a1, a2, a3, a4]


# Пример вызова make_suite
suite = make_suite("Trithemus", "LCG", "SP Net", "CCM")
print("Generated Suite:", suite)
print(suite[0])
print(suite[1][0][0])


Generated Suite: [[<function RoundKeys.wrap_CHCLCGM_next at 0x7eb796f8a3b0>, 8], [[<function SPNetwork.block_SP_frw at 0x7eb796f8b910>, <function SPNetwork.block_SP_inv at 0x7eb796f8b9a0>], 8], [<function TrithemusCipher.frw_S_TrithemusM at 0x7eb796f88430>, <function TrithemusCipher.inv_S_TrithemusM at 0x7eb796f88310>], [<function CCMP_frw at 0x7eb7973b2b90>, <function CCMP_inv at 0x7eb7973b29e0>]]
[<function RoundKeys.wrap_CHCLCGM_next at 0x7eb796f8a3b0>, 8]
<function SPNetwork.block_SP_frw at 0x7eb796f8b910>


In [ ]:
# СЛАЙД 64 СХОД blockxor и combine
print("blockxor(КОНЬ, А__Г) =", Utilities.blockxor("КОНЬ", "А__Г"))
print("blockxor(КОНЬ, АБВГ) =", Utilities.blockxor("КОНЬ", "АБВГ"))
print("blockxor(КОНЬ, ЛУНЬ) =", Utilities.blockxor("КОНЬ", "ЛУНЬ"))
print("blockxor(КААА, АБВГ) =", Utilities.blockxor("КААА", "АБВГ"))

blockxor(КОНЬ, А__Г) = ЙОНЧ
blockxor(КОНЬ, АБВГ) = ЙММЧ
blockxor(КОНЬ, ЛУНЬ) = ЖЫ__
blockxor(КААА, АБВГ) = ЙВБД


In [ ]:
ASSOCDATA_ARRAY[1]

'"ВБ" "АЛИСА_АЖ" "БОБ___ОЧ" "ЕГИПТЯНИН"'

In [ ]:
# СЛАЙД 68 СХОД
AD = ["ВБ", "АЛИСА_АЖ", "БОБ___ОЧ", "ЕГИПТЯНИН", "АБВГД"]
PACKET = [AD, "БОБ_НЕМНОГО_ПЬЯН", INPUTS_ARRAY[0], ""]

SUIT = (SPNetwork.block_SP_frw, TrithemusCipher.frw_S_TrithemusM, 8)
keyset = RoundKeys.produce_round_keys("СЕАНСОВЫЙ_КЛЮЧИК", 8, SUIT[1], RoundKeys.wrap_CHCLCGM_next)

# Прямая CCM
Q_TEST1m = SPNetwork.CCM_frw(PACKET, keyset, 0, SUIT)
print("Q_TEST1m (зашифрованное):", Q_TEST1m)
print(f"Q_TEST1m[3] = {Q_TEST1m[3]}")
print(f"Q_TEST1m[2] = {Q_TEST1m[2]}")

print("----------------------------------------------------------------------------------------")

# Обратная CCM
R_TEST1m = SPNetwork.CCM_inv(Q_TEST1m, keyset, 0, SUIT)
print("R_TEST1m (расшифрованное):", R_TEST1m)
print(f"R_TEST1m[3] = {R_TEST1m[3]}")
print(f"R_TEST1m[2] = {R_TEST1m[2]}")

Q_TEST1m (зашифрованное): [['ВБ', 'АЛИСА_АЖ', 'БОБ___ОЧ', 'ЕГИПТЯНИН', 'АБВГД'], 'БОБ_НЕМНОГО_ПЬЯН', 'ЦИ_ЙБВТЛРАВГФЫЭСАМНХЖЮХЦОЯЧСЗ_КЗЕИКЯСЙЧХБМНМЮЖХЦЗЙЬПЗИДКОПМЬИТНЩХДФЗШЯЭЯМЕИШЧСТЭЬЮФНЩ__ЦПЦЬОИКЯТСБОПЖЕРЖЗЬМЖЯМУФЕЮЖДФХРВДЕРАКССПИХРЫМЩМ_ННЭНМГПАЮЧБЙШОЫГЦТЮЛ_ЭНУЫВУГКБМИД__МЧЗЛУЭМЯ_РБПФЕНМУЬУЩНЛИОМРЖМЫКТЖХФЗЭЕЯНЮВЕУЮЛЕТЧЙЭЬОФУГФЛУЛГЯСЗООВПИУМТЫЩТЗЮЬЧЦУЕЩСЩЦЙБ_ЖДЧАГТРЫБПЮЧКЮТВШУЭЖПКЩЧТПЧЛАНЛЧУЗИИЦВ_УГЙЖЗСКЮБХ_ТНВЗРЯЬЭЯЗЬРЖМЙПЙОСЧЦЮШ_ПЦЧШЭЙТШИЧБЮЧЮ_ИРШШХЬЬ_ВЗГШЕЭГЕЩКСД', 'ПШБПЕСЫН_ЗГЖФПЭШ']
Q_TEST1m[3] = ПШБПЕСЫН_ЗГЖФПЭШ
Q_TEST1m[2] = ЦИ_ЙБВТЛРАВГФЫЭСАМНХЖЮХЦОЯЧСЗ_КЗЕИКЯСЙЧХБМНМЮЖХЦЗЙЬПЗИДКОПМЬИТНЩХДФЗШЯЭЯМЕИШЧСТЭЬЮФНЩ__ЦПЦЬОИКЯТСБОПЖЕРЖЗЬМЖЯМУФЕЮЖДФХРВДЕРАКССПИХРЫМЩМ_ННЭНМГПАЮЧБЙШОЫГЦТЮЛ_ЭНУЫВУГКБМИД__МЧЗЛУЭМЯ_РБПФЕНМУЬУЩНЛИОМРЖМЫКТЖХФЗЭЕЯНЮВЕУЮЛЕТЧЙЭЬОФУГФЛУЛГЯСЗООВПИУМТЫЩТЗЮЬЧЦУЕЩСЩЦЙБ_ЖДЧАГТРЫБПЮЧКЮТВШУЭЖПКЩЧТПЧЛАНЛЧУЗИИЦВ_УГЙЖЗСКЮБХ_ТНВЗРЯЬЭЯЗЬРЖМЙПЙОСЧЦЮШ_ПЦЧШЭЙТШИЧБЮЧЮ_ИРШШХЬЬ_ВЗГШЕЭГЕЩКСД
----------------------------------------------------------------------------------------
R_TE

# Проверка CCM


In [ ]:
AD = ["ВБ", "БОБ___ЬЬ", "АЛИСА_ЯЗ", "ЭКЛАМПСИЯ"]
MESSAGES = INPUTS_ARRAY
SUITE = make_suite("Trithemus", "LCG", "SP Net", "CCM")
CHANNEL = CCM(AD, MESSAGES, "СЕАНСОВЫЙ_КЛЮЧИК", SUITE, "СЕМИХАТОВ_КВАНТЫ", "send")

KeyError: 0